In [30]:
import os
import ollama
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [31]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [32]:
# Let's just make sure the model is loaded

!ollama pull llama3.2:1b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manif

In [33]:
# class to represent a Webpage
# Some websites need you to use proper headers when fetching them:

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
ed = Website("https://case.edu.pk/department-of-computer-science/")
print(ed.title)
print(ed.text)

In [36]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [37]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [38]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [39]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [41]:
summarize("https://case.edu.pk/department-of-computer-science/")

"### Department of Computer Science - Sir Syed CASE Institute of Technology Overview\n\nThe Department of Computer Science at Sir Syed_CASE Institute of Technology is a leading academic unit that offers undergraduate, graduate, and postgraduate programs in various disciplines of computer science. Here's a short summary of the department:\n\n**Admissions:**\n- Bachelor's Programs are offered for Pre-Medical students.\n- Students with Pre-Medical backgrounds can take admission to Computing programs without appearing in Board intermediate level exams.\n\n### Faculty Portal\n\nThe faculty members of the Department of Computer Science have undergone rigorous training and research, providing high-quality education to students. The faculty is equipped with extensive experience and knowledge in their respective areas of specialization.\n\n### Research & Labs\n\nThe department has a strong focus on research and development, with state-of-the-art laboratories and equipment for various computatio

In [42]:
 # function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [44]:
display_summary("https://case.edu.pk/department-of-computer-science/")

### Department of Computer Science - Sir Syed CASE Institute of Technology

#### About Us

The Department of Computer Science at the Sir Syed CASE Institute of Technology (SS CASE IT) is a premier institution in Islamabad, offering undergraduate degrees in five domains: Computer Science, Electrical Engineering, Management Sciences, Accounting, and Business Analytics.

#### Chancellor's Message

As per an order from the Federal Government Act of Majlis-e-Shoora (Parliament) No. F.9(26)/2018-Legis dated May 22, 2018, Sir Syed CASE Institute of Technology has been transformed into **Sir Syed CASE Institute of Technology** (SS CASE IT). The institute is now a degree-awarding institution under the Federal Government Act.

#### Engineering Education Trust

The Department of Computer Science is part of the Engineering Education Trust, which ensures that the courses offered are accredited and meet certain standards.

#### Vice Chancellor's Message

Under the new act, Sir Syed CASE Institute of Technology has moved to its newly constructed campus located at Block A, Multi Garden, Sector B-17, Islamabad. The institute offers five programs in Electrical Engineering, Computer Engineering, Software Engineering, and Computer Science at the Bachelor, Master, and PhD levels.

#### Academic Calendar

The academic calendar is available on the website, indicating courses offered and their respective start dates.

### Code of Conduct

A code of conduct has been established to ensure that students adhere to certain standards of behavior in the institution's educational environment.

### Academic Rules & Policies

Regulations governing the institution's policies are outlined, including rules for student conduct, academic integrity, and other relevant matters.

#### Faculty Portal

The faculty portal is accessible via a link on the website. This allows students to contact their respective faculty members or seek guidance on various academic issues.

### Library

A comprehensive library is available at the institute, offering a wide range of texts, journals, and online resources for research purposes.

#### Quality Enhancement Cell

A quality enhancement cell has been established to ensure that the institution meets certain standards of quality in its educational programs.